# **02 | Data Preprocessing Notebook**
## **Notebook Overview**
- **Author:** Trice Balthazar
- **Date:** 2025-12-03
- **Version:** 1.0
- **Purpose:** Prepare clean data for analysis and visualization, based on project objectives.

## **Table of Contents**
1. Setup & Imports
2. Load Clean Data
3. Preprocessing Steps
4. Objective Alignment
5. Summary & Export

# **1. Setup & Imports**

In [3]:
import pandas as pd
import numpy as np

# Create constant(s)
START_TIME = pd.to_datetime('2025-04-10 18:15:00', format='%Y-%m-%d %H:%M:%S') # Start time of online conference

# **2. Load Clean Data**

In [4]:
# Set folder path
folder_path = r"..\data\clean"

# Import clean zoom data
zoom_df = pd.read_csv(folder_path+r'\clean-zoom-data.csv')

# Convert join_time and leave_time to pd.DateTime()
zoom_df['join_time'] = pd.to_datetime(zoom_df['join_time'], format='%Y-%m-%d %H:%M:%S')
zoom_df['leave_time'] = pd.to_datetime(zoom_df['leave_time'],format='%Y-%m-%d %H:%M:%S')

# Import clean eventbrite data
eventbrite_df = pd.read_csv(folder_path+r'\clean-eventbrite-data.csv')

# **3. Pre-processing**

In [5]:
# Merge data sets
merged = eventbrite_df.merge(
    zoom_df[['email']],           # keep only email from zoom to avoid column collisions
    on='email',
    how='outer',                  # keep union to see all combos
    indicator=True
)

# merged.head()

## **3.1. Create Subsets for Further Analysis**

In [6]:
# registered AND attended
registered_and_attended = merged[merged['_merge'] == 'both'].copy()

# registered AND did NOT attend (registered only)
registered_not_attended = merged[merged['_merge'] == 'left_only'].copy()

# attended AND did NOT register (attended only) -- useful extra subset
attended_not_registered = merged[merged['_merge'] == 'right_only'].copy()

# Create one DataFrame for all attendees, regardeless of registration
all_attendees = pd.concat([registered_and_attended, attended_not_registered], axis=0)
all_attendees = all_attendees.reset_index(drop=True)

In [7]:
# Save counts for each case
registered_and_attended_count = len(registered_and_attended)
attended_not_registered_count = len(attended_not_registered)
all_attendees_count = len(all_attendees)
registered_not_attended_count = len(registered_not_attended)

# Display values
print(f'registered & attended: {registered_and_attended_count}')
print(f'not registered & attended: {attended_not_registered_count}')
print(f'total attendees: {all_attendees_count}')
print(f'registered & not attended: {registered_not_attended_count}')

registered & attended: 448
not registered & attended: 158
total attendees: 606
registered & not attended: 773


## **3.2. Eventbrite Data Pre-processing**

In [8]:
# Remove duplicates
registered_and_attended.drop_duplicates(subset=["email", "city", "province_territory", "please_specify"], inplace=True)

# Create DataFrame for Canada only. Remove nulls and blanks from province_territory column
registered_and_attended_can = registered_and_attended[registered_and_attended["province_territory"].notna() & (registered_and_attended["province_territory"] != "")].copy()
registered_and_attended_can.reset_index(drop=True, inplace=True)

# Create DataFrame for International only. Keep non-null values in please_specify column.
# International attendees specified their cities in the 'please_specify' column, therefore we only care about the non-blank rows.
registered_and_attended_int = registered_and_attended[registered_and_attended["please_specify"].notna()].copy()
registered_and_attended_int.reset_index(drop=True, inplace=True)

### **3.2.1 Attendance by Location**

In [9]:
# Group by province and count instances
registered_and_attended_can_province = registered_and_attended_can.groupby(by='province_territory', as_index=False).agg(count=('province_territory', 'count'))

# Sort value in descending order
registered_and_attended_can_province.sort_values('count', ascending=False, inplace=True)

# Remove '(residing outside of Canada)
registered_and_attended_can_province['province_territory'] = registered_and_attended_can_province['province_territory'].str.split('(').str[0].str.strip()

registered_and_attended_can_province

,province_territory,count
5,ON,322
2,International,10
6,QC,7
3,MB,6
0,AB,3
1,BC,3
4,NS,2
7,SK,1


In [10]:
# Replace US, USA, California and Florida with United States
registered_and_attended_int['please_specify'] = registered_and_attended_int['please_specify'].str.replace(r'\bUS\b','United States', regex=True)
registered_and_attended_int['please_specify'] = registered_and_attended_int['please_specify'].str.replace(r'\bUSA\b','United States', regex=True)
registered_and_attended_int['please_specify'] = registered_and_attended_int['please_specify'].str.replace(r'\bCalifornia\b','United States', regex=True)
registered_and_attended_int['please_specify'] = registered_and_attended_int['please_specify'].str.replace(r'\bFlorida\b','United States', regex=True)
# registered_and_attended_int.head()

In [11]:
# Group by countries (please_specify column) and count
registered_and_attended_int_countries = registered_and_attended_int.groupby(by='please_specify', as_index=False).agg(count=('please_specify', 'count'))
# Sort values
registered_and_attended_int_countries.sort_values(by='count', ascending=False, inplace=True)

registered_and_attended_int_countries

,please_specify,count
1,United States,7
0,United Kingdom,1
2,na,1


## **3.2. Zoom Data Pre-processing**

In [12]:
# Aggregate: first join (min) and last leave (max)
agg_span = (zoom_df.groupby('email', as_index=False).agg(first_join=('join_time', 'min'), # keep min. join time
                                                         last_leave =('leave_time', 'max'), # keep max. leave time
                                                         session_count = ('join_time', 'size'))  # number of records for each email
)

# Compute duration in conference
agg_span['time_in_session'] = np.ceil(((agg_span['last_leave'] - agg_span['first_join']).dt.total_seconds() / 60)) # Convert time delta to minutes and round to the nearest minute

# Compute join_minute
agg_span['join_minute'] = np.ceil(((agg_span['first_join'] - START_TIME).dt.total_seconds() / 60))
# Compute left_minute
agg_span['leave_minute'] = np.ceil(((agg_span['last_leave'] - START_TIME).dt.total_seconds() / 60))

# Create a mask for attendee(s) who joined before the start time
join_before_start_time_mask = agg_span['join_minute'] < 0

# Remove attendee(s) who joined before the start time
agg_span = agg_span[~join_before_start_time_mask]

# Remove unnecessary columns
agg_span = agg_span.drop(columns=['email', 'first_join', 'last_leave', 'session_count'])
agg_span.head()

,time_in_session,join_minute,leave_minute
0,156.0,14.0,170.0
1,114.0,57.0,170.0
2,141.0,29.0,169.0
3,32.0,25.0,57.0
4,143.0,16.0,159.0


# **4. Objective Alignment**

## 4.1. Merge Datasets on Email  
Merging the Zoom and Eventbrite datasets on the **email** column allows us to compare who registered, who attended, and how these groups overlap.

## 4.2. Identify Key Attendance Subsets  
Using the `_merge` column from the merge results:
- **Registered and Attended** (`both`): People who appear in both datasets.
- **Registered and Did Not Attend** (`left_only`) :Emails found only in the Eventbrite data.
- **Did Not Register and Attended** (`right_only`): Emails found only in the Zoom attendance data.

These subsets help understand overall engagement and audience behavior.

## 4.3. Preprocess the “Registered and Attended” Subset  
This subset is used for analyzing where attendees are located.
- Supports building **bar charts** for Canadian provinces and international countries.
- Ensures clean, accurate data when breaking down attendance by location.

## 4.4. Resolve Multiple Join/Leave Records  
Some attendees join and leave multiple times in Zoom logs. To create a consistent view:
- **Group by email** and keep:
  - the **earliest join time**  
  - the **latest leave time**  
- This gives a reasonable estimate of each attendee’s total time in the session.

## 4.5. Calculate Time in Session  
Compute:  
**`time_in_session = leave_time - join_time`**  
This provides the duration (in minutes) each attendee stayed.

# **5. Export & Sumamry**

## **5.1 Export Preprocessed Data**

In [13]:
# Export pre-processed zoom data
agg_span.to_csv(r'..\data\preprocessed\zoom-session-data.csv', index=False)

# Export pre-processed eventbrite data
registered_and_attended_int_countries.to_csv(r'..\data\preprocessed\international-attendee-data.csv', index=False)
registered_and_attended_can_province.to_csv(r'..\data\preprocessed\attendee-location-data.csv', index=False)

## **5.2 Summary**

In this notebook, the registration and attendance datasets were merged on **email** to understand participant behavior. From this merge, three groups were identified:

- **Registered & Attended** — emails found in both datasets  
- **Registered & Did Not Attend** — emails found only in the registration data  
- **Attended Without Registering** — emails found only in the attendance data  

The **Registered & Attended** group was then prepared for further analysis. This included:

- Using the dataset to create **location-based breakdowns** of attendees across Canadian provinces and international countries.
- Resolving multiple join/leave timestamps by **grouping by email** and keeping the **earliest join** and **latest leave** times.
- Calculating each attendee’s **time in session**:
  - join times  
  - leave times  
  - total time in session  

This preprocessing ensures the dataset is clean, consistent, and ready for visualization and insights.